# questions / possible problems:
- when we filter out "egg", does our python-code also filter out "Egg Beaters egg substitute". if so, we need to change our python-code.



# 0. load data and setup

In [1]:
import pandas as pd
import numpy as np


# change display settings to show all columns
pd.set_option('display.max_columns', None)

In [2]:
# the data is from https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews/data

reviews = pd.read_csv("data/reviews.csv")
print("shape is", reviews.shape)
reviews.head(3)

shape is (1401982, 8)


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z


In [3]:
# the data is from https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews/data

df = pd.read_csv("data/recipes.csv")
print("shape is", df.shape)
df.head(3)

shape is (522517, 28)


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."


# 1. EDA

In [23]:
def get_unique_values(column):
    if pd.api.types.is_integer_dtype(column):  # Überprüfe, ob der Datentyp eine Ganzzahl ist
        unique_vals = sorted(set(column.dropna()))
        min_val, max_val = column.min(), column.max()
        if unique_vals == list(range(min_val, max_val + 1)):
            return f"(range:{min_val}-{max_val})"
        return unique_vals
    return sorted(set(column.dropna()))

def summary(df=df):
    summary_df = pd.DataFrame({
        'variable id': range(1,df.shape[1]+1),
        'dtypes': df.dtypes.astype(str),
        'missing count': df.isna().sum(),
        'missing ratio': round(df.isna().sum() / df.shape[0],2),
#        'values': [get_unique_values(df[col]) for col in df.columns],
        'unique': df.nunique() #datatype float if NaN in df
    })
    return summary_df

summary(df).sort_values(by=['dtypes', 'variable id'])


,variable id,dtypes,missing count,missing ratio,unique
AggregatedRating,15,float64,61133,0.49,9
ReviewCount,16,float64,59772,0.48,199
Calories,17,float64,0,0.00,15091
FatContent,18,float64,0,0.00,2266
SaturatedFatContent,19,float64,0,0.00,1078
CholesterolContent,20,float64,0,0.00,3125
SodiumContent,21,float64,0,0.00,21790
CarbohydrateContent,22,float64,0,0.00,4469
FiberContent,23,float64,0,0.00,791
SugarContent,24,float64,0,0.00,3625


In [20]:
summary(reviews)

,variable id,dtypes,missing count,missing ratio,unique
ReviewId,1,int64,0,0.0,1401982
RecipeId,2,int64,0,0.0,271678
AuthorId,3,int64,0,0.0,271907
AuthorName,4,object,0,0.0,241365
Rating,5,int64,0,0.0,6
Review,6,object,214,0.0,1392745
DateSubmitted,7,object,0,0.0,1384268
DateModified,8,object,0,0.0,1384268


---
# 2. filter vegan recipes
---

## 2.1 filter out recipes with animal products 

In [4]:
# 1.try
non_vegan_ingredients = {"abalone", "alligator meat", "alligator sausage", "alligator tail steaks", "American cheese", "anchovy", "anchovy fillet", "anchovy fillets", "anchovy paste", "andouille chicken sausage", "andouille sausage", "andouille turkey sausage", "Anejo cheese", "asadero cheese", "asiago cheese", "back bacon", "bacon", "bacon bits", "bacon drippings", "bacon fat", "bacon grease", "bagoong", "beef", "beef bones with marrow", "beef bouillon", "beef brisket", "beef broth", "beef burger", "beef chuck", "beef drippings", "beef eye round", "beef fat", "beef flank steak", "beef kielbasa", "beef liver", "beef prosciutto", "beef rib", "beef roast", "beef schnitzel", "beef shank", "beef short ribs", "beef sirloin", "beef steak", "beef suet", "beef sweetbreads", "beef tenderloin", "beefsteak", "bleu cheese", "blue cheese", "boneless beef", "boneless chicken", "boneless duck", "boneless ham", "boneless lamb", "boneless pork", "boneless salmon", "bottarga", "bouillon", "braunschweiger sausage", "bresaola", "brie cheese", "broth", "buffalo mozzarella", "bulk Italian sausage", "bulk pork sausage", "bulk sausage", "butter", "buttermilk", "cabrales cheese", "caciocavallo", "camembert cheese", "Canadian bacon", "Canadian cheddar cheese", "capacola ham", "cheddar cheese", "cheese", "chicken", "chicken bouillon", "chicken broth", "chicken fat", "chicken giblets", "chicken gizzard", "chicken liver", "chicken meat", "clam", "clams", "codfish", "colby cheese", "condensed beef broth", "condensed chicken broth", "cooked corned beef", "cooked duck", "cooked ham", "cooked lamb", "cooked pork", "corned beef", "cottage cheese", "crab", "crabmeat", "cream", "cream cheese", "creme fraiche", "duck", "duck fat", "duck liver", "dulce de leche", "egg", "eggs", "eel", "fish", "fish sauce", "fontina cheese", "foie gras", "gelatin", "ghee", "goat cheese", "goose", "gorgonzola", "gouda cheese", "gruyere cheese", "ham", "heavy cream", "herring", "havarti cheese", "honey", "ice cream", "jamon serrano", "kidney suet", "kielbasa", "lamb", "lard", "lobster", "mascarpone cheese", "meat", "milk", "monterey jack cheese", "mozzarella cheese", "muenster cheese", "octopus", "oyster", "pancetta", "parmesan cheese", "pecorino cheese", "pheasant", "pork", "prosciutto", "provolone cheese", "quail", "quark", "rabbit", "ricotta cheese", "romano cheese", "roquefort cheese", "salmon", "sardine", "sausage", "scallop", "shrimp", "sour cream", "squid", "suet", "swiss cheese", "tallow", "trout", "tuna", "turkey", "veal", "venison", "whey", "whipped cream", "whole milk", "yogurt",

# 2.try-Version1 (2.try is after 1. try)        
"abalone", "aioli", "albacore tuna", "albacore tuna in water", "alfredo sauce", "anchovy fillet", "anchovy fillets", "baby beef liver", "baby chicken", "baby chickens", "baby clams", "baby shrimp", "bacon", "bacon-meatloaf", "bangers", "barbecued chicken", "barbecued pork", "bay scallop", "bay scallops", "bay shrimp", "beef bouillon cube", "beef bouillon cubes", "beef bouillon granules", "beef bouillon powder", "beef burgers", "beef chuck with bone", "beef mince", "beef ribs", "beef round rump roast", "beef round steak", "beef round tip steak", "beef round tip steaks", "beef short ribs with bones", "beef silverside", "beef sirloin steak", "beef sirloin steaks", "beef steaks", "beef stew meat", "beef tenderloin steak", "beef tenderloin steaks", "beef tip roast", "beef top round steak", "beef broth", "bleu cheese", "bleu cheese spread", "blue cheese", "bocconcini", "bone dust", "boned lamb", "boneless atlantic salmon fillet", "boneless beef brisket", "boneless beef chuck", "boneless beef chuck roast", "boneless beef chuck shoulder pot roast", "boneless beef cube", "boneless beef cubes", "boneless beef roast", "boneless beef round steak", "boneless beef rump roast", "boneless beef short rib", "boneless beef short ribs", "boneless beef sirloin", "boneless beef top loin steak", "boneless beef top loin steaks", "boneless beef top round", "boneless beef top round steak", "boneless beef top round steaks", "boneless beef top sirloin steak", "boneless beef top sirloin steaks", "boneless boston pork roast", "boneless bottom round roast", "boneless center cut pork chop", "boneless center cut pork chops", "boneless center cut pork loin roast", "boneless chicken breast", "boneless chicken breast half", "boneless chicken breast halves", "boneless chicken breasts", "boneless duck breast", "boneless duck breast halves", "boneless duck breasts", "boneless duck meat", "boneless fresh pork", "boneless lamb chops", "boneless lamb loin", "boneless lamb roast", "boneless lamb shoulder", "boneless lean lamb", "boneless lean pork", "boneless leg of lamb", "boneless pork blade roast", "boneless pork butt shank steaks", "boneless pork chop", "boneless pork chops", "boneless pork cutlet", "boneless pork cutlets", "boneless pork filet", "boneless pork filets", "boneless pork leg", "boneless pork loin", "boneless pork loin chop", "boneless pork loin chops", "boneless pork loin roast", "boneless pork loin steaks", "boneless pork ribs", "boneless pork roast", "boneless pork shoulder", "boneless pork sirloin", "boneless pork top loin", "boneless round steak", "boneless salmon fillet", "boneless salmon fillets", "boneless sirloin tip roast", "boneless skinless chicken", "boneless skinless chicken breast", "boneless skinless chicken breast half", "boneless skinless chicken breast halves", "boneless skinless chicken breasts", "boneless skinless chicken thighs", "boneless skinless cod", "boneless skinless duck breast halves", "boneless skinless duck breasts", "boneless skinned chicken breast", "boneless skinned chicken breasts", "bonito flakes", "bonito stock", "bonito tuna", "bouillon cube", "bouillon cubes", "bratwurst", "breast milk", "brick cheese", "broiler chicken", "broiler chickens", "broiler-fryer chicken", "broiler-fryer chickens", "burrata cheese", "butter", "butter beans", "buttermilk", "butterscotch pudding", "calf liver", "caerphilly cheese", "camembert cheese", "canadian bacon", "canned anchovy fillet", "canned anchovy fillets", "canned baby clams", "canned chicken", "canned clams", "canned crab", "canned flaked ham", "canned salmon", "canned shrimp", "canned tuna", "catfish fillet", "catfish fillets", "catfish meat", "cheddar cheese", "cheese slice", "cheese slices", "cheese spread", "chicken back", "chicken backs", "chicken bouillon cube", "chicken bouillon cubes", "chicken bouillon granule", "chicken bouillon granules", "chicken bouillon mix", "chicken bouillon powder", "chicken breast", "chicken breast fillet", "chicken breast fillets", "chicken breast half", "chicken breast halve", "chicken breast halves", "chicken breasts", "chicken broth", "chicken cutlet", "chicken cutlets", "chicken drumstick", "chicken drumsticks", "chicken feet", "chicken fillet", "chicken fillets", "chicken gizzards", "chicken halves", "chicken leg", "chicken leg meat", "chicken legs", "chicken legs-thighs", "chicken livers", "chicken neck", "chicken necks", "chicken parts", "chicken piece", "chicken pieces", "chicken portions", "chicken tenderloins", "chicken thigh", "chicken thigh fillet", "chicken thigh fillets", "chicken thigh pieces", "chicken thighs", "chicken wings", "chinese barbecue pork", "chinese barbecue sauce", "chinese barbecued duck", "chorizo", "chorizo sausage", "chunk light tuna", "chunk tuna", "clam broth", "clam chowder", "clams in shell", "cod", "cod fish fillet", "cod fish fillets", "cod steaks", "codfish roe", "colby cheese", "colby-monterey jack cheese", "comte cheese", "cooked ham", "country ham", "country pork strips", "country sausage", "country-style boneless pork ribs", "country-style pork ribs", "crab meat", "crawfish", "crawfish fat", "crawfish meat", "crawfish tail", "crawfish tail meat", "crawfish tails", "crayfish", "crayfish tail", "crayfish tails", "cream", "cream cheese", "cream cheese spread", "cream cheese with chives", "cream cheese with green onion", "creamed honey", "creme fraiche", "dolcelatte", "double gloucester cheese", "duck breast", "duck breast halves", "duck breasts", "duck carcass", "duck carcasses", "duck confit", "duck fat", "duck hindquarters", "duck legs", "duckling", "ducklings", "ducks", "edam cheese", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "eggs", "farmers cheese", "feta", "feta cheese", "fish bouillon cube", "fish bouillon cubes", "fish stock", "flank steak", "fontina", "fontina cheese", "fryer chicken", "fryer chickens", "frying chicken", "frying chickens", "gelatin", "goat cheese", "gorgonzola", "gruyere", "gruyere cheese", "haddock fillet", "haddock fillets", "halibut fillet", "halibut fillets", "halibut steaks", "ham", "ham bone", "ham bones", "ham bouillon cube", "ham fat", "ham hock", "ham hocks", "ham shank", "ham shanks", "ham slices", "ham steak", "ham steaks", "hard-boiled egg", "hard-boiled eggs", "herring fillet", "herring fillets", "herring in wine sauce", "herring roe", "herring tidbits", "hot dogs", "ice cream", "italian sausage", "jarlsberg cheese", "kielbasa", "lamb", "lamb baby food", "lamb backstrap", "lamb backstraps", "lamb blade chops", "lamb bone", "lamb bones", "lamb breast", "lamb chop", "lamb chops", "lamb cutlets", "lamb fillet", "lamb fillets", "lamb gravy", "lamb knuckles", "lamb leg chops", "lamb leg steak", "lamb leg steaks", "lamb liver", "lamb loin chop", "lamb loin chops", "lamb mince", "lamb neck", "lamb necks", "lamb rack", "lamb racks", "lamb rib chops", "lamb ribs", "lamb shank", "lamb shanks", "lamb shoulder", "lamb sirloin", "lamb steak", "lamb steaks", "lamb stew meat", "lamb stock", "lamb stock cube", "lamb tenderloins", "lean bacon", "lean baked ham", "lean beef", "lean beef chuck", "lean beef chuck roast", "lean beef round", "lean boneless lamb", "lean boneless leg of lamb", "lean ground beef", "lean ground lamb", "lean ground turkey", "lean ham", "lean hamburger", "lean lamb", "lean lamb fillets", "lean lamb leg steak", "lean lamb pieces", "lean lamb steaks", "lean lamb stew meat", "lean round steak", "lean rump steak", "lean salt pork", "limburger cheese", "liver", "lobster head", "lobster meat", "lobster tail", "lobster tails", "lobsters", "loin lamb", "london broil beef", "lump crabmeat", "mackerel fillet", "mackerel fillets", "manchego cheese", "mascarpone", "mayonnaise", "maytag blue cheese", "meatballs", "mimolette cheese", "monterey jack cheese", "mozzarella cheese", "muenster cheese", "mussels", "new england clam chowder", "non-fat milk", "ocean perch", "oysters", "pancetta", "paneer cheese", "parma ham", "parmesan cheese", "partridge breasts", "partridges", "pecorino cheese", "pecorino romano cheese", "pepperoni", "perch", "pheasant breast", "pheasants", "pork", "pork belly", "pork bouillon cube", "pork broth", "pork butt", "pork chop", "pork chops", "pork cutlet", "pork cutlets", "pork fat", "pork jowl", "pork loin", "pork loin chops", "pork ribs", "pork roast", "pork sausage", "pork sausage link", "pork sausage links", "pork shoulder", "pork tenderloin", "pork tenderloins", "prosciutto", "prosciutto di parma", "prosciutto ham", "provolone cheese", "quail", "quail eggs", "queso blanco", "queso fresco", "rabbit", "raclette cheese", "red snapper", "red snapper fillet", "red snapper fillets", "refried beans with bacon", "rendered duck fat", "ricotta cheese", "robiola cheese", "roast beef", "roasting chicken", "roasting chickens", "rock cod fish", "roquefort blue cheese", "roquefort cheese", "salmon", "salmon fillet", "salmon fillets", "salmon roe", "salmon steak", "salmon steaks", "sardine fillet", "sardine fillets", "sausage", "sausage links", "scallops", "scamorza cheese", "sea bass", "sea bass fillet", "sea bass fillets", "sea scallops", "shrimp", "shrimp bouillon cube", "shrimp bouillon cubes", "shrimp paste", "sirloin beef", "sirloin lamb chops", "sirloin tip roast", "sirloin tip steak", "sirloin tip steaks", "skirt steak", "smoked back bacon", "smoked bacon", "smoked ham", "smoked ham hock", "smoked ham hocks", "smoked kielbasa", "smoked salmon", "smoked sausage", "smoked trout", "smoked trout fillet", "smoked trout fillets", "sole fillet", "sole fillets", "sour cream", "steamer clam", "steamer clams", "steelhead trout", "stewing beef", "stewing chicken", "stewing lamb", "stilton cheese", "sweetbreads", "swiss cheese", "swordfish fillets", "swordfish steak", "swordfish steaks", "taleggio", "tasso", "tilapia", "tilapia fillet", "tilapia fillets", "tiger shrimp", "tripe", "trout fillet", "trout fillets", "tuna", "tuna fillet", "tuna fillets", "tuna fish", "tuna in brine", "tuna in olive oil", "tuna in vegetable oil", "tuna in water", "tuna packed in oil", "tuna salad", "tuna steak", "tuna steaks", "turkey", "turkey bacon", "turkey breast", "turkey breast tenderloin", "turkey breast tenderloin steaks", "turkey breast tenderloins", "turkey broth", "turkey carcass", "turkey fat", "turkey kielbasa", "turkey meat", "turkey parts", "turkey polish kielbasa", "turkey sausage", "turkey slice", "turkey slices", "turkey steaks", "turkey stuffing", "turkey tenderloins", "veal", "veal broth", "veal demi-glace", "veal sweetbreads", "wensleydale cheese", "whipped cream", "whiting fish fillets", "whole chicken", "whole chickens", "whole milk", "whole milk cottage cheese", "whole milk mozzarella", "whole milk ricotta cheese", "wild ducks", "yogurt",

# 2.try-Version2. (2.try-Version2 is  a subset von 2.try-Version1)
"abalone", "aioli", "albacore tuna", "albacore tuna in water", "alfredo sauce", "anchovy fillet", "anchovy fillets", "baby beef liver", "baby chicken", "baby chickens", "baby clams", "baby shrimp", "bacon", "bacon-meatloaf", "bangers", "barbecued chicken", "barbecued pork", "bay scallop", "bay scallops", "bay shrimp", "beef bouillon cube", "beef bouillon cubes", "beef bouillon granules", "beef bouillon powder", "beef burgers", "beef chuck with bone", "beef mince", "beef ribs", "beef round rump roast", "beef round steak", "beef round tip steak", "beef round tip steaks", "beef short ribs with bones", "beef silverside", "beef sirloin steak", "beef sirloin steaks", "beef steaks", "beef stew meat", "beef tenderloin steak", "beef tenderloin steaks", "beef tip roast", "beef top round steak", "beef broth", "bleu cheese", "bleu cheese spread", "blue cheese", "bocconcini", "bone dust", "boned lamb", "boneless atlantic salmon fillet", "boneless beef brisket", "boneless beef chuck", "boneless beef chuck roast", "boneless beef chuck shoulder pot roast", "boneless beef cube", "boneless beef cubes", "boneless beef roast", "boneless beef round steak", "boneless beef rump roast", "boneless beef short rib", "boneless beef short ribs", "boneless beef sirloin", "boneless beef top loin steak", "boneless beef top loin steaks", "boneless beef top round", "boneless beef top round steak", "boneless beef top round steaks", "boneless beef top sirloin steak", "boneless beef top sirloin steaks", "boneless boston pork roast", "boneless bottom round roast", "boneless center cut pork chop", "boneless center cut pork chops", "boneless center cut pork loin roast", "boneless chicken breast", "boneless chicken breast half", "boneless chicken breast halves", "boneless chicken breasts", "boneless duck breast", "boneless duck breast halves", "boneless duck breasts", "boneless duck meat", "boneless fresh pork", "boneless lamb chops", "boneless lamb loin", "boneless lamb roast", "boneless lamb shoulder", "boneless lean lamb", "boneless lean pork", "boneless leg of lamb", "boneless pork blade roast", "boneless pork butt shank steaks", "boneless pork chop", "boneless pork chops", "boneless pork cutlet", "boneless pork cutlets", "boneless pork filet", "boneless pork filets", "boneless pork leg", "boneless pork loin", "boneless pork loin chop", "boneless pork loin chops", "boneless pork loin roast", "boneless pork loin steaks", "boneless pork ribs", "boneless pork roast", "boneless pork shoulder", "boneless pork sirloin", "boneless pork top loin", "boneless round steak", "boneless salmon fillet", "boneless salmon fillets", "boneless sirloin tip roast", "boneless skinless chicken", "boneless skinless chicken breast", "boneless skinless chicken breast half", "boneless skinless chicken breast halves", "boneless skinless chicken breasts", "boneless skinless chicken thighs", "boneless skinless cod", "boneless skinless duck breast halves", "boneless skinless duck breasts", "boneless skinned chicken breast", "boneless skinned chicken breasts", "bonito flakes", "bonito stock", "bonito tuna", "bouillon cube", "bouillon cubes", "bratwurst", "breast milk", "brick cheese", "broiler chicken", "broiler chickens", "broiler-fryer chicken", "broiler-fryer chickens", "burrata cheese", "butter", "butter beans", "buttermilk", "butterscotch pudding", "calf liver", "caerphilly cheese", "camembert cheese", "canadian bacon", "canned anchovy fillet", "canned anchovy fillets", "canned baby clams", "canned chicken", "canned clams", "canned crab", "canned flaked ham", "canned salmon", "canned shrimp", "canned tuna", "catfish fillet", "catfish fillets", "catfish meat", "cheddar cheese", "cheese slice", "cheese slices", "cheese spread", "chicken back", "chicken backs", "chicken bouillon cube", "chicken bouillon cubes", "chicken bouillon granule", "chicken bouillon granules", "chicken bouillon mix", "chicken bouillon powder", "chicken breast", "chicken breast fillet", "chicken breast fillets", "chicken breast half", "chicken breast halve", "chicken breast halves", "chicken breasts", "chicken broth", "chicken cutlet", "chicken cutlets", "chicken drumstick", "chicken drumsticks", "chicken feet", "chicken fillet", "chicken fillets", "chicken gizzards", "chicken halves", "chicken leg", "chicken leg meat", "chicken legs", "chicken legs-thighs", "chicken livers", "chicken neck", "chicken necks", "chicken parts", "chicken piece", "chicken pieces", "chicken portions", "chicken tenderloins", "chicken thigh", "chicken thigh fillet", "chicken thigh fillets", "chicken thigh pieces", "chicken thighs", "chicken wings", "chinese barbecue pork", "chinese barbecue sauce", "chinese barbecued duck", "chorizo", "chorizo sausage", "chunk light tuna", "chunk tuna", "clam broth", "clam chowder", "clams in shell", "cod", "cod fish fillet", "cod fish fillets", "cod steaks", "codfish roe", "colby cheese", "colby-monterey jack cheese", "comte cheese", "cooked ham", "country ham", "country pork strips", "country sausage", "country-style boneless pork ribs", "country-style pork ribs", "crab meat", "crawfish", "crawfish fat", "crawfish meat", "crawfish tail", "crawfish tail meat", "crawfish tails", "crayfish", "crayfish tail", "crayfish tails", "cream", "cream cheese", "cream cheese spread", "cream cheese with chives", "cream cheese with green onion", "creamed honey", "creme fraiche", "dolcelatte", "double gloucester cheese", "duck breast", "duck breast halves", "duck breasts", "duck carcass", "duck carcasses", "duck confit", "duck fat", "duck hindquarters", "duck legs", "duckling", "ducklings", "ducks", "edam cheese", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "eggs", "farmers cheese", "feta", "feta cheese", "fish bouillon cube", "fish bouillon cubes", "fish stock", "flank steak", "fontina", "fontina cheese", "fryer chicken", "fryer chickens", "frying chicken", "frying chickens", "gelatin", "goat cheese", "gorgonzola", "gruyere", "gruyere cheese", "haddock fillet", "haddock fillets", "halibut fillet", "halibut fillets", "halibut steaks", "ham", "ham bone", "ham bones", "ham bouillon cube", "ham fat", "ham hock", "ham hocks", "ham shank", "ham shanks", "ham slices", "ham steak", "ham steaks", "hard-boiled egg", "hard-boiled eggs", "herring fillet", "herring fillets", "herring in wine sauce", "herring roe", "herring tidbits", "hot dogs", "ice cream", "italian sausage", "jarlsberg cheese", "kielbasa", "lamb", "lamb baby food", "lamb backstrap", "lamb backstraps", "lamb blade chops", "lamb bone", "lamb bones", "lamb breast", "lamb chop", "lamb chops", "lamb cutlets", "lamb fillet", "lamb fillets", "lamb gravy", "lamb knuckles", "lamb leg chops", "lamb leg steak", "lamb leg steaks", "lamb liver", "lamb loin chop", "lamb loin chops", "lamb mince", "lamb neck", "lamb necks", "lamb rack", "lamb racks", "lamb rib chops", "lamb ribs", "lamb shank", "lamb shanks", "lamb shoulder", "lamb sirloin", "lamb steak", "lamb steaks", "lamb stew meat", "lamb stock", "lamb stock cube", "lamb tenderloins", "lean bacon", "lean baked ham", "lean beef", "lean beef chuck", "lean beef chuck roast", "lean beef round", "lean boneless lamb", "lean boneless leg of lamb", "lean ground beef", "lean ground lamb", "lean ground turkey", "lean ham", "lean hamburger", "lean lamb", "lean lamb fillets", "lean lamb leg steak", "lean lamb pieces", "lean lamb steaks", "lean lamb stew meat", "lean round steak", "lean rump steak", "lean salt pork", "limburger cheese", "liver", "lobster head", "lobster meat", "lobster tail", "lobster tails", "lobsters", "loin lamb", "london broil beef", "lump crabmeat", "mackerel fillet", "mackerel fillets", "manchego cheese", "mascarpone", "mayonnaise", "maytag blue cheese", "meatballs", "mimolette cheese", "monterey jack cheese", "mozzarella cheese", "muenster cheese", "mussels", "new england clam chowder", "non-fat milk", "ocean perch", "oysters", "pancetta", "paneer cheese", "parma ham", "parmesan cheese", "partridge breasts", "partridges", "pecorino cheese", "pecorino romano cheese", "pepperoni", "perch", "pheasant breast", "pheasants", "pork", "pork belly", "pork bouillon cube", "pork broth", "pork butt", "pork chop", "pork chops", "pork cutlet", "pork cutlets", "pork fat", "pork jowl", "pork loin", "pork loin chops", "pork ribs", "pork roast", "pork sausage", "pork sausage link", "pork sausage links", "pork shoulder", "pork tenderloin", "pork tenderloins", "prosciutto", "prosciutto di parma", "prosciutto ham", "provolone cheese", "quail", "quail eggs", "queso blanco", "queso fresco", "rabbit", "raclette cheese", "red snapper", "red snapper fillet", "red snapper fillets", "refried beans with bacon", "rendered duck fat", "ricotta cheese", "robiola cheese", "roast beef", "roasting chicken", "roasting chickens", "rock cod fish", "roquefort blue cheese", "roquefort cheese", "salmon", "salmon fillet", "salmon fillets", "salmon roe", "salmon steak", "salmon steaks", "sardine fillet", "sardine fillets", "sausage", "sausage links", "scallops", "scamorza cheese", "sea bass", "sea bass fillet", "sea bass fillets", "sea scallops", "shrimp", "shrimp bouillon cube", "shrimp bouillon cubes", "shrimp paste", "sirloin beef", "sirloin lamb chops", "sirloin tip roast", "sirloin tip steak", "sirloin tip steaks", "skirt steak", "smoked back bacon", "smoked bacon", "smoked ham", "smoked ham hock", "smoked ham hocks", "smoked kielbasa", "smoked salmon", "smoked sausage", "smoked trout", "smoked trout fillet", "smoked trout fillets", "sole fillet", "sole fillets", "sour cream", "steamer clam", "steamer clams", "steelhead trout", "stewing beef", "stewing chicken", "stewing lamb", "stilton cheese", "sweetbreads", "swiss cheese", "swordfish fillets", "swordfish steak", "swordfish steaks", "taleggio", "tasso", "tilapia", "tilapia fillet", "tilapia fillets", "tiger shrimp", "tripe", "trout fillet", "trout fillets", "tuna", "tuna fillet", "tuna fillets", "tuna fish", "tuna in brine", "tuna in olive oil", "tuna in vegetable oil", "tuna in water", "tuna packed in oil", "tuna salad", "tuna steak", "tuna steaks", "turkey", "turkey bacon", "turkey breast", "turkey breast tenderloin", "turkey breast tenderloin steaks", "turkey breast tenderloins", "turkey broth", "turkey carcass", "turkey fat", "turkey kielbasa", "turkey meat", "turkey parts", "turkey polish kielbasa", "turkey sausage", "turkey slice", "turkey slices", "turkey steaks", "turkey stuffing", "turkey tenderloins", "veal", "veal broth", "veal demi-glace", "veal sweetbreads", "wensleydale cheese", "whipped cream", "whiting fish fillets", "whole chicken", "whole chickens", "whole milk", "whole milk cottage cheese", "whole milk mozzarella", "whole milk ricotta cheese", "wild ducks", "yogurt", "live lobster", "live lobsters",

# 3.try-1.Version
"97% fat-free cooked ham", "ahi", "ahi tuna steaks", "Alpine Lace 97% reduced-fat cooked ham", "Alpine Lace reduced-fat swiss cheese", "anchovies", "andouille sausages", "Appenzeller cheese", "asiago-parmesan cheese", "baby swiss cheese", "Bacardi light rum", "bacon", "bacon-meatloaf", "beef flavored Rice-A-Roni", "beef machaca", "beef stew meat", "bel paese cheese", "best end of neck welsh lamb", "black caviar", "black cod steaks", "bleu cheese", "blue cheese", "bottom round beef roast", "bottom round steak", "bottom round steaks", "boursin cheese", "breakfast sausage", "brie", "brown and serve sausages", "burata cheese", "butter", "butter flavor shortening", "butter lettuce", "butter pecan ice cream", "Butterball boneless breast of turkey", "buttermilk", "buttermilk baking mix", "carne asada", "carnaroli rice", "cheddar and american blend cheese", "cheddar and colby cheese", "cheddar cheese cube", "cheddar cheese cubes", "cheddar cheese powder", "cheese", "cheshire cheese", "chicken gumbo soup", "chicken style seasoning", "chihuahua cheese", "Chinese barbecue pork", "Chinese barbecued duck", "clam juice", "clams", "colby", "condensed chicken gumbo soup", "condensed milk", "corned beef brisket", "cottage cheese", "cottage roll", "crab", "crabmeat", "cream", "cream cheese", "cream sherry", "creamy peanut butter", "deli corned beef", "deli ham", "deli roast beef", "double Gloucester cheese", "duck", "duck fat", "egg", "egg beaters", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "eggs", "Emmental cheese", "evaporated milk", "eye of round roast", "farmer cheese", "fat free cheese", "fat free cream cheese", "fat-free cottage cheese", "fat-free cream cheese", "filet of beef", "filet of fresh sea bass", "fillet of sole", "fillets of sole", "fish", "fish sauce", "flank of beef", "Fontina cheese", "fresh crab", "fresh crabmeat", "fresh crabs", "fresh lump crabmeat", "fresh mozzarella ball", "fresh mozzarella balls", "fresh mozzarella cheese", "fresh salmon", "fresh scallops", "fresh sea scallop", "fresh sea scallops", "fresh shrimp", "fresh small shrimp", "fresh swordfish steaks", "fresh tuna", "fresh tuna filets", "frozen crab", "frozen crabmeat", "frozen lobster tail", "frozen lobster tails", "frozen shrimp", "gelatin", "goat cheese", "gorgonzola", "gorgonzola dolce blue cheese", "gouda cheese", "Greek feta cheese", "ground beef", "ground beef round", "ground chicken", "ground lamb", "ground pork", "ground turkey", "gruyere cheese", "gulf shrimp", "ham", "hamburger", "hamburger meat", "hard-shelled clams", "havarti with caraway", "havarti with dill", "heavy cream", "heavy whipping cream", "honey", "honey graham crackers", "honey syrup", "honey-molasses", "honey-roasted cashews", "ice cream", "Italian pork sausage", "Italian sausage", "jarlsberg cheese", "joint of beef", "jumbo lump crab meat", "kasseri cheese", "king salmon", "kielbasa", "lamb", "lamb chop", "lamb chops", "lancashire cheese", "lard", "Limburger cheese", "littleneck clams", "low fat cottage cheese", "low-fat cheddar cheese", "low-fat cream cheese", "low-fat milk", "low-fat mozzarella cheese", "low-fat ricotta cheese", "low-fat swiss cheese", "low-sodium ham", "mahi mahi", "mahi mahi fillet", "mahi mahi fillets", "masago smelt roe", "milk", "milk solids", "monterey jack and cheddar cheese blend", "monterey jack cheese", "monterey jack pepper cheese", "mozzarella", "mozzarella cheddar blend cheese", "mozzarella cheese cubes", "mozzarella cheese with sun-dried tomatoes and basil", "mozzarella string cheese", "mozzarella-cheddar blend cheese", "mozzarella-parmesan cheese blend", "mozzarella-provolone cheese blend", "non-fat cottage cheese", "parmesan", "parmesan cheese", "part-skim cottage cheese", "part-skim mozzarella cheese", "part-skim ricotta cheese", "pecorino cheese", "pork", "pork belly", "pork chop", "pork chops", "pork loin", "pork shoulder", "pork tenderloin", "poultry", "prosciutto", "provolone cheese", "quail eggs", "queso anejo", "rack of lamb", "racks of lamb", "red salmon", "reduced fat romano cheese", "ricotta", "ricotta cheese", "ricotta salata", "romano cheese", "Roquefort cheese", "salmon", "salmon fillet", "sardines", "sausage", "sausage meat", "sausages", "scallops", "serrano ham", "sharp cheddar cheese", "sheep's milk cheese", "shellfish", "shrimp", "skim milk", "smoked cheddar cheese", "smoked gouda cheese", "smoked gruyere cheese", "smoked salmon", "sole", "sour cream", "stilton cheese", "swiss cheese", "tuna", "turkey", "turkey breast", "turkey meat", "veal", "whipped cream", "whipped cream cheese", "whole milk", "yogurt",

# 3.try-2.Version
"1% fat buttermilk", "1% fat cottage cheese", "1% low-fat chocolate milk", "1% low-fat milk", "2% buttermilk", "2% cheddar cheese", "2% evaporated milk", "2% fat cottage cheese", "2% large-curd cottage cheese", "2% low-fat chocolate milk", "2% low-fat milk", "2% milk", "2% mozzarella cheese", "20% sour cream", "3-cheese gourmet cheddar blend cheese", "5% fat ricotta cheese", "97% fat-free cooked ham", "aged cheddar cheese", "aged white cheddar cheese", "ahi", "ahi tuna steaks", "Alpine Lace 97% reduced-fat cooked ham", "Alpine Lace reduced-fat swiss cheese", "amaretti di saronno cookies", "anchovies", "andouille sausages", "Appenzeller cheese", "asiago-parmesan cheese", "baby swiss cheese", "bacon", "bacon-meatloaf", "beef flavored Rice-A-Roni", "beef machaca", "beef stew meat", "bel paese cheese", "best end of neck welsh lamb", "black caviar", "black cod steaks", "bleu cheese", "blue cheese", "bone dust", "bottom round beef roast", "bottom round steak", "bottom round steaks", "boursin cheese", "breakfast sausage", "brie", "brown and serve sausages", "burata cheese", "butter", "butter flavor shortening", "butter lettuce", "butter pecan ice cream", "Butterball boneless breast of turkey", "buttermilk", "buttermilk baking mix", "carne asada", "cheddar and american blend cheese", "cheddar and colby cheese", "cheddar cheese cube", "cheddar cheese cubes", "cheddar cheese powder", "cheese", "cheshire cheese", "chicken gumbo soup", "chicken stock", "chicken style seasoning", "chihuahua cheese", "Chinese barbecue pork", "Chinese barbecued duck", "clam juice", "clams", "colby", "condensed chicken gumbo soup", "condensed milk", "corned beef brisket", "cottage cheese", "cottage roll", "crab", "crabmeat", "cream", "cream cheese", "cream sherry", "cream-style cottage cheese", "creamy peanut butter", "cultured buttermilk", "deli corned beef", "deli ham", "deli roast beef", "double Gloucester cheese", "duck", "duck fat", "egg", "egg beaters", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "eggs", "Emmental cheese", "evaporated milk", "eye of round roast", "farmer cheese", "fat free cheese", "fat free cream cheese", "fat-free buttermilk", "fat-free cheddar cheese", "fat-free cottage cheese", "fat-free cream cheese", "fat-free evaporated milk", "fat-free ham", "fat-free mayonnaise", "fat-free milk", "fat-free monterey jack cheese", "fat-free parmesan cheese", "fat-free ricotta cheese", "fat-free sour cream", "fat-free swiss cheese", "filet of beef", "filet of fresh sea bass", "fillet of sole", "fillets of sole", "fish", "fish sauce", "flank of beef", "Fontina cheese", "fresh crab", "fresh crabmeat", "fresh crabs", "fresh lump crabmeat", "fresh mozzarella ball", "fresh mozzarella balls", "fresh mozzarella cheese", "fresh salmon", "fresh scallops", "fresh sea scallop", "fresh sea scallops", "fresh shrimp", "fresh small shrimp", "fresh swordfish steaks", "fresh tuna", "fresh tuna filets", "frozen crab", "frozen crabmeat", "frozen lobster tail", "frozen lobster tails", "frozen shrimp", "full-cream milk", "full-fat milk", "gelatin", "goat cheese", "gorgonzola", "gorgonzola dolce blue cheese", "gouda cheese", "Greek feta cheese", "ground beef", "ground beef round", "ground chicken", "ground lamb", "ground pork", "ground turkey", "gruyere cheese", "gulf shrimp", "half & half light cream", "half and half milk", "half cream", "half-and-half", "half-and-half cream", "halloumi cheese", "ham", "hamburger", "hamburger meat", "hard-shelled clams", "havarti with caraway", "havarti with dill", "heavy cream", "heavy sweet cream", "heavy whipping cream", "herbed cheddar cheese", "honey", "honey graham crackers", "honey syrup", "honey-molasses", "honey-roasted cashews", "homogenized milk", "ice cream", "Italian pork sausage", "Italian sausage", "jarlsberg cheese", "joint of beef", "jumbo lump crab meat", "kasseri cheese", "king salmon", "kielbasa", "lamb", "lamb chop", "lamb chops", "lancashire cheese", "lard", "large curd cottage cheese", "large-curd cottage cheese", "light butter", "light cheddar cheese", "light cheese", "light cottage cheese", "light cream cheese", "light jarlsberg cheese", "light mayonnaise", "light monterey jack cheese", "light mozzarella cheese", "light sour cream", "Limburger cheese", "littleneck clams", "low fat cottage cheese", "low-fat buttermilk", "low-fat cheddar cheese", "low-fat cream cheese", "low-fat kielbasa", "low-fat milk", "low-fat mozzarella cheese", "low-fat ricotta cheese", "low-fat sour cream", "low-fat swiss cheese", "low-moisture part-skim mozzarella cheese", "low-salt ham", "low-sodium ham", "mahi mahi", "mahi mahi fillet", "mahi mahi fillets", "masago smelt roe", "milk", "milk solids", "monterey jack and cheddar cheese blend", "monterey jack cheese", "monterey jack pepper cheese", "monterey jack-style soy cheese", "mozzarella", "mozzarella cheddar blend cheese", "mozzarella cheese cubes", "mozzarella cheese with sun-dried tomatoes and basil", "mozzarella string cheese", "mozzarella-cheddar blend cheese", "mozzarella-parmesan cheese blend", "mozzarella-provolone cheese blend", "non-fat beef broth", "non-fat cheddar cheese", "non-fat chicken broth", "non-fat cottage cheese", "non-fat milk", "non-fat parmesan cheese", "non-fat sour cream", "nonfat beef broth", "nonfat cheddar cheese", "nonfat chicken broth", "nonfat cottage cheese", "nonfat milk", "nonfat parmesan cheese", "nonfat sour cream", "parmesan", "parmesan cheese", "part-skim cottage cheese", "part-skim farmer cheese", "part-skim mozzarella cheese", "part-skim ricotta cheese", "pecorino cheese", "pecorino pepato cheese", "pork", "pork belly", "pork chop", "pork chops", "pork loin", "pork shoulder", "pork tenderloin", "poultry", "processed cheese", "processed cheese food", "processed cheese spread", "processed cheese with jalapeno peppers", "processed swiss cheese", "prosciutto", "prosciutto di Parma", "provolone cheese", "quail eggs", "quark", "queso anejo", "rack of lamb", "racks of lamb", "red salmon", "reduced fat romano cheese", "reduced-fat cheddar cheese", "reduced-fat cream cheese", "reduced-fat extra sharp cheddar cheese", "reduced-fat jarlsberg cheese", "reduced-fat kielbasa", "reduced-fat milk", "reduced-fat monterey jack cheese", "reduced-fat mozzarella cheese", "reduced-fat ricotta cheese", "reduced-fat sharp cheddar cheese", "reduced-fat sour cream", "reduced-fat swiss cheese", "ricotta", "ricotta cheese", "ricotta salata", "romano cheese", "Roquefort cheese", "salmon", "salmon fillet", "sardines", "sausage", "sausage meat", "sausages", "scallops", "serrano ham", "sharp American cheese", "sharp cheddar cheese", "sharp white cheddar cheese", "sheep's milk cheese", "shellfish", "shrimp", "skim evaporated milk", "skim milk", "skim milk buttermilk", "skim milk powder", "skim milk ricotta cheese", "skim milk yogurt", "smoked cheddar cheese", "smoked cod", "smoked gouda cheese", "smoked gruyere cheese", "smoked link sausage", "smoked mozzarella cheese", "smoked pork jowl", "smoked salmon", "smoked streaky bacon", "sockeye salmon", "sole", "sour cream", "sour cream substitute", "spring chicken", "Stilton cheese", "sweet creamy butter", "sweet Italian sausage", "sweet milk", "swiss cheese", "tuna", "turkey", "turkey breast", "turkey escalopes", "turkey meat", "turkey Polish kielbasa", "unsalted butter", "veal", "Vermont sharp cheddar cheese", "weisswurst sausage", "weisswurst sausages", "white cheddar cheese", "white meat chicken", "white turkey meat", "whipped butter", "whipped cream", "whipped cream cheese", "whole buttermilk", "whole milk", "yoghurt", "yogurt", "yogurt cheese",

# 4.try-1.Version
"Absolutely the Best New York Cheesecake", "Bacon-Meatloaf", "bacon", "bear shaped graham crackers", "beef Machaca", "beef", "blachan", "Bombay duck", "bouillon cube", "bouillon cubes", "bouillon granules", "butter lettuce", "butter", "buttermilk", "canned chicken broth", "canned clam juice", "canned crab meat", "cheese Whiz", "cheese", "chicken broth", "chicken gumbo soup", "chicken meat", "chicken sausage", "chicken stock", "chicken", "clam juice", "clams", "condensed milk", "cottage cheese", "crab meat", "cream cheese", "cream", "creme fraiche", "cuttlefish", "dairy", "demi-glace", "dried beef", "dried bonito flakes", "dried fish", "dried shrimp", "egg beaters", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "egg", "eggs", "fat pork", "fatty bacon", "fatty pork", "fish sauce", "fish stock", "fish", "gelatin", "ground beef", "ground chicken", "ground lamb", "ground meat", "ground pork", "ground turkey", "ground veal", "ham", "heavy cream", "honey", "hot dogs", "ice cream", "kielbasa", "lamb chop", "lamb chops", "lamb", "lard", "milk powder", "milk", "mozzarella cheese", "mozzarella", "oyster sauce", "parmesan cheese", "parmesan", "pork bacon", "pork belly", "pork chop", "pork chops", "pork fat", "pork hock", "pork loin", "pork", "poultry seasoning", "prosciutto", "quark", "raw chicken", "ricotta cheese", "ricotta", "roast beef", "roast turkey", "salmon", "salted butter", "sausage", "shrimp paste", "shrimp", "smoked bacon", "sour cream", "squid", "steak", "sweet cream", "tuna", "turkey breast", "turkey", "unsalted butter", "veal", "whipped cream", "whipping cream", "whole milk", "yogurt",

# 4.try-2.Version
"Absolutely the Best New York Cheesecake", "Bacon-Meatloaf", "bacon", "bear shaped graham crackers", "beef Machaca", "beef", "blachan", "blue cheese", "Bombay duck", "bone dust", "bouillon cube", "bouillon cubes", "bouillon granules", "butter lettuce", "butter", "buttermilk", "canned chicken broth", "canned clam juice", "canned crab meat", "carnation evaporated milk", "cheese Whiz", "cheese", "chicken bouillon", "chicken broth", "chicken gumbo soup", "chicken meat", "chicken sausage", "chicken stock", "chicken", "clam juice", "clams", "condensed milk", "cottage cheese", "crab meat", "cream cheese", "cream", "creme fraiche", "custard", "cuttlefish", "dairy", "demi-glace", "dried beef", "dried bonito flakes", "dried fish", "dried shrimp", "egg beaters", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "egg", "eggs", "fat free sour cream", "fat pork", "fatty bacon", "fatty pork", "fish sauce", "fish stock", "fish", "gelatin", "graham crackers", "ground beef", "ground chicken", "ground lamb", "ground meat", "ground pork", "ground turkey", "ground veal", "ham", "heavy cream", "honey", "hot dogs", "ice cream", "italian sausage", "kielbasa", "lamb chop", "lamb chops", "lamb", "lard", "low fat sour cream", "marshmallow cream", "marshmallow creme", "marshmallow", "marshmallows", "milk powder", "milk", "mozzarella cheese", "mozzarella", "non-fat sour cream", "oyster sauce", "parmesan cheese", "parmesan", "pepperoni", "pork bacon", "pork belly", "pork chop", "pork chops", "pork fat", "pork hock", "pork loin", "pork sausage", "pork", "poultry seasoning", "prosciutto", "quark", "raw chicken", "ricotta cheese", "ricotta", "roast beef", "roast turkey", "salami", "salmon", "salted butter", "sausage", "shrimp paste", "shrimp", "smoked bacon", "sour cream", "squid", "steak", "sweet cream", "tuna", "turkey breast", "turkey", "unsalted butter", "veal", "whipped cream", "whipping cream", "whole milk", "yogurt",

"Absolute the Best New York Cheesecake", "Absolutely the Best New York Cheesecake (Gluten-Free)", "American cheese", "bacon", "Bacardi 151 rum", "beef", "beef bouillon", "beef broth", "beef chunk", "beef chunks", "beef steak", "beef stock", "beef wieners", "bite-size fresh mozzarella cheese balls", "blue cheese", "boston lettuce", "butter", "butter lettuce", "Carnation Evaporated Milk", "Carnation instant milk", "cheddar cheese", "cheese", "cheesecake", "chicken", "chicken bouillon", "chicken broth", "chicken gumbo soup", "chicken meat", "chicken piece", "chicken pieces", "chicken stock", "clam", "clams", "crab meat", "cream cheese", "custard", "dairy", "egg", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "eggs", "fish", "fish sauce", "gelatin", "ground beef", "ground meat", "ham", "heavy cream", "honey", "ice cream", "kielbasa", "lamb", "lamb chop", "lamb chops", "lard", "mascarpone", "mayonnaise", "meat", "meatballs", "milk", "milk powder", "milk solid", "mozzarella", "mozzarella cheese", "parmesan", "parmesan cheese", "Pet milk", "Philadelphia Cream Cheese", "pork", "pork chops", "pork hocks", "pork loin", "pork roast", "pork sausage", "provolone cheese", "quark", "ricotta", "ricotta cheese", "salmon", "sausage", "scallops", "shrimp", "swiss cheese", "tuna", "tuna fillet", "tuna fillets", "tuna steaks", "turkey", "turkey breast", "turkey meat", "whey", "whipped cream", "whole milk", "yogurt",

"Absolutely the Best New York Cheesecake", "active dry yeast", "Almesan - Vegan Parmesan", "american cheese", "anchovy", "Bacardi dark rum", "bacon", "bacon bits", "bacon fat", "beef", "beef bouillon", "beef broth", "beef chunk", "beef chunks", "beef pieces", "beef stock", "blue cheese", "butter", "buttermilk", "canadian bacon", "cheddar cheese", "cheese", "cheese sauce", "cheese whiz", "chicken", "chicken bouillon", "chicken broth", "chicken gumbo soup", "chicken meat", "chicken piece", "chicken pieces", "chicken stock", "clam", "clams", "condensed milk", "Cool Whip", "cottage cheese", "crab meat", "cream", "cream cheese", "creme fraiche", "dried beef", "duck", "egg", "egg beaters", "egg substitute", "egg white", "egg whites", "egg yolk", "egg yolks", "eggs", "fish", "fish sauce", "gelatin", "goat cheese", "ground beef", "ground pork", "ham", "heavy cream", "honey", "ice cream", "italian sausage", "jumbo shrimp", "lamb", "lamb chop", "lamb chops", "lard", "liquid eggs", "milk", "miso", "monterey jack cheese", "mozzarella", "mozzarella cheese", "parmesan", "parmesan cheese", "parmigiano", "parmigiano-reggiano", "pheasant", "pork", "pork sausage", "provolone cheese", "quark", "rabbit", "ricotta", "roast beef", "romano cheese", "salmon", "sausage", "scungilli", "shrimp", "sour cream", "squid", "steak", "stilton cheese", "swiss cheese", "tuna", "turkey", "turkey meat", "venison", "whipped cream", "whipping cream", "whole milk", "worcestershire sauce", "yogurt",

# i don't know why i can not filter this one???
"Absolutely the Best New York Cheesecake (Gluten-Free)"
}

In [5]:
df[df['RecipeIngredientParts']=="Absolutely the Best New York Cheesecake (Gluten-Free)"]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions


In [6]:
# Ella wrote this code-snipped with claude.ai and Ella has not understood all of the code yet. Understanding this code-snipped is on Ellas to-do-list. Ella also needs to check the UserWarning below this code-snipped


def filter_non_vegan_recipes(df, non_vegan_ingredients):
    def contains_non_vegan(ingredient_str):
        # Remove the c() wrapper and split into individual ingredients
        ingredients = ingredient_str.replace('c(', '').replace(')', '').split(',')
        # Clean up each ingredient
        ingredients = [ing.strip().replace('"', '') for ing in ingredients]
        
        # Check for exact matches with non_vegan_ingredients
        return any(ing in non_vegan_ingredients for ing in ingredients)
    
    # Create a mask for rows to keep (those without non-vegan ingredients)
    vegan_mask = ~df['RecipeIngredientParts'].apply(contains_non_vegan)

    print(f"Original number of recipes: {len(df)}")
    print(f"Number of vegan recipes: {len(df[vegan_mask])}")
    print(f"Percentage of vegan recipes: {(len(df[vegan_mask]) / len(df) * 100):.2f}%")
    
    return df[vegan_mask]

df = filter_non_vegan_recipes(df, non_vegan_ingredients)
df.head(3)

Original number of recipes: 522517
Number of vegan recipes: 124160
Percentage of vegan recipes: 23.76%


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


## 2.2. check if 'Egg Beaters egg substitute'-entries still exists

In [7]:
egg_mask = df['RecipeIngredientParts'].str.contains(r'\bEgg Beaters egg substitute\b', case=False, regex=True, na=False)

df[egg_mask][['Name', 'RecipeIngredientParts']].head()

,Name,RecipeIngredientParts
9703,Yummy Low Fat Chocolate Chip Pound Cake,"c(""instant chocolate pudding mix"", ""applesauce..."
15529,Pumpkin Spice Cake,"c(""pumpkin"", ""Egg Beaters egg substitute"")"
29208,11&quot; Brownie Tart Dough,"c(""all-purpose flour"", ""baking powder"", ""salt""..."
34790,Low Fat Sugar Free Banana Muffins,"c(""bananas"", ""Splenda sugar substitute"", ""Egg ..."
41889,Chinese Hot and Sour Soup Lite,"c(""fat-free chicken broth"", ""soy sauce"", ""shii..."


## 2.3 only allow recipes with laben "Vegan"

In [8]:
#select recipes with keywords "vegan"
df_vegan_label = df[df['Keywords'].str.contains("Vegan", case=False, na=False)]   #it makes no difference if we write Vegan or vegan

print("shape is", df_vegan_label.shape)
df_vegan_label.head(3)

shape is (16981, 28)


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."
50,90,Almond Paste,1537,Barbara5,PT168H,PT15M,PT168H15M,1999-08-23T12:33:00Z,Make and share this Almond Paste recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Dessert,"c(""Fruit"", ""Nuts"", ""Lactose Free"", ""Vegan"", ""L...","c(""1"", ""3"", ""1"", ""2"")","c(""fresh lemon juice"", ""water"", ""sugar"")",4.0,2.0,2123.5,119.8,9.0,0.0,49.1,244.0,22.6,210.7,48.9,NaN,2 pounds,"c(""Force blanched almonds through fine blade o..."
88,133,Blueberry Pancake Syrup,1571,Linda Johansen,PT36M,PT50M,PT1H26M,1999-08-06T00:41:00Z,"I've made this blueberry syrup several times, ...",character(0),Sauces,"c(""Breakfast"", ""Dessert"", ""Berries"", ""Fruit"", ...","c(""4"", ""3"", ""2"", ""3"", NA)","c(""blueberries"", ""water"", ""lemons, rind of"", ""...",5.0,4.0,1329.7,1.0,0.1,0.0,16.6,342.8,7.1,328.9,2.2,NaN,2 pints,"c(""Pour the blueberries into a saucepan and cr..."


## 2.4 make a list of all ingridients (that are in df) - and store them in a txt-file

In [9]:
df.shape

(124160, 28)

In [10]:
import re
from tqdm import tqdm  # für den Fortschrittsbalken


# Funktion zum Extrahieren der Zutaten aus dem R-Format String
def extract_ingredients(text):
    if pd.isna(text):  # Überprüfe auf NaN-Werte
        return []
    # Entferne c() und extrahiere die einzelnen Zutaten
    matches = re.findall(r'"([^"]*)"', text)
    return matches

# Initialisiere eine leere Set für unique Zutaten
all_ingredients = set()

# Verarbeite den DataFrame in Chunks
chunk_size = 10000
num_chunks = len(df) // chunk_size + 1

for i in tqdm(range(num_chunks)):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    
    # Verarbeite jede Zeile im aktuellen Chunk
    chunk = df.iloc[start_idx:end_idx]
    
    # Füge die Zutaten zum Set hinzu
    for ingredients_str in chunk['RecipeIngredientParts']:
        ingredients = extract_ingredients(ingredients_str)
        all_ingredients.update(ingredients)

# Konvertiere das Set in eine sortierte Liste
unique_ingredients = sorted(list(all_ingredients), key=str.lower)



# Zeige die ersten 10 Zutaten und die Gesamtanzahl
print(f"Anzahl einzigartiger Zutaten: {len(unique_ingredients)}")
print("\nBeispiel (erste 10 Zutaten):")
print(unique_ingredients[:50])

# Optional: Speichere die Liste in einer Datei
with open('unique_ingredients.txt', 'w', encoding='utf-8') as f:
    for ingredient in unique_ingredients:
        f.write(ingredient + '\n')

100%|██████████| 13/13 [00:00<00:00, 95.39it/s]

Anzahl einzigartiger Zutaten: 4702

Beispiel (erste 10 Zutaten):
['&iquest;porque No? Mexican Pesto', '&quot;sun-Dried&quot; Tomatoes', '10-inch corn tortillas', '10-inch flour tortillas', '10-minute Success rice', '100 proof vodka', "2bleu's 2 Minute 2 Easy Pizza Sauce", "2bleu's Sweet Mustard Sauce for Pretzels and More!", '3 Legume Butter', '4 grain bread flour', '6-inch corn tortilla', '6-inch corn tortillas', '6-inch flour tortillas', '6-inch tortillas', '7-inch corn tortillas', '7-inch flour tortillas', '8-inch corn tortillas', '8-inch fat-free flour tortillas', '8-inch flour tortillas', '9-inch graham cracker crust', '9-inch graham cracker crusts', 'absinthe', 'Absolut citron vodka', 'Absolut kurant vodka', 'Absolut mandarin vodka', 'Absolut vodka', 'Absolutely the Best New York Cheesecake (Gluten-Free)', 'Accent seasoning', 'achar', 'Achiote Paste', 'acini di pepe pasta', 'acorn squash', 'adobo sauce', 'adobo seasoning', 'Advieh', 'adzuki beans', 'aeroplane vegetarian jelly cry

---
# 3. data-preprocessing

[guide for data-preprocessing](https://medium.com/@maleeshadesilva21/preprocessing-steps-for-natural-language-processing-nlp-a-beginners-guide-d6d9bf7689c9)

---


## 3.1 Convert to Lowercase

In [40]:
string_columns = df.select_dtypes(include=['object']).columns.tolist()
string_columns.remove('Images')

for column in string_columns:
    df[column] = df[column].str.lower()

df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,low-fat berry blue frozen dessert,1533,dancer,pt24h,pt45m,pt24h45m,1999-08-09t21:46:00z,make and share this low-fat berry blue frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",frozen desserts,"c(""dessert"", ""low protein"", ""low cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""toss 2 cups berries with sugar."", ""let stan..."
2,40,best lemonade,1566,stephen little,pt5m,pt30m,pt35m,1999-09-05t19:52:00z,this is from one of my first good house keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",beverages,"c(""low protein"", ""low cholesterol"", ""healthy"",...","c(""1 1/2"", ""1"", na, ""1 1/2"", na, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""into a 1 quart jar with tight fitting lid, ..."
4,42,cabbage soup,1538,duckie067,pt30m,pt20m,pt50m,1999-09-19t06:19:00z,make and share this cabbage soup recipe from f...,"""https://img.sndimg.com/food/image/upload/w_55...",vegetable,"c(""low protein"", ""vegan"", ""low cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""mix everything together and bring to a boil..."
8,46,a jad - cucumber pickle,1533,dancer,NaN,pt25m,pt25m,1999-08-11t19:48:00z,make and share this a jad - cucumber pickle re...,character(0),vegetable,"c(""thai"", ""asian"", ""free of..."", ""< 30 mins"")","c(""1/2"", ""5"", ""2"", ""1"", ""1"", ""1"")","c(""rice vinegar"", ""haeo"")",5.0,2.0,4.3,0.0,0.0,0.0,0.7,1.1,0.2,0.2,0.1,NaN,1 cup,"c(""slice the cucumber in four lengthwise, then..."
14,52,cafe cappuccino,2178,troyh,NaN,pt5m,pt5m,1999-08-31t21:05:00z,make and share this cafe cappuccino recipe fro...,"c(""https://img.sndimg.com/food/image/upload/w_...",beverages,"c(""low cholesterol"", ""high protein"", ""healthy""...","c(""1/2"", ""3/4"", ""1"", ""1/2"")","c(""instant coffee"", ""sugar"", ""nonfat dry milk ...",5.0,1.0,62.2,0.1,0.0,1.3,36.6,12.8,0.0,11.8,2.7,18.0,2 1/4 cups,"c(""stir ingredients together."", ""process in a ..."



## 3.2 Removal of Punctuations
## 3.3 Removal of Stopwords
## 3.4 Removal of Frequent Words
## 3.5 Removal of Rare Words
## 3.6 Removal of Special Characters
## 3.7 Stemming
## 3.8 Lemmatization & POS Tagging
## 3.9 Removal of URLs
## 3.10 Removal of HTML Tags
## 3.11 Spelling Correction